# В этом файле будем делать алгоритм коллаборативной фильтрации
 входом должен быть id юзера, в ответ получаем рекомендации

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [5]:
cosine_list = []
with open('cosine_dim.txt') as file:
    for line in tqdm(file):
        temp = line.split()
        temp = int(temp[0]), int(temp[1]), float(temp[2])
        cosine_list.append(temp)

cosine_dim = pd.DataFrame(columns=['x', 'y', 'cosine'], data=(cosine_list))

0it [00:00, ?it/s]

796274it [00:01, 604496.69it/s]


In [9]:
len(cosine_dim)

796274

In [8]:
cosine_dim.to_excel('cosine_dim.xlsx')

In [6]:
cosine_dim[cosine_dim.x == 0].sort_values('cosine', ascending=False)

,x,y,cosine
47,0,25581,0.982789
120,0,19022,0.980321
76,0,58950,0.963732
151,0,28052,0.961028
122,0,10873,0.956637
...,...,...,...
182,0,28790,0.625483
274,0,39187,0.593576
222,0,29798,0.592464
323,0,16258,0.564709


In [14]:
# Все существующие пользователи по порядку
all_users = sorted(set(cosine_dim.x).union(set(cosine_dim.y)))

In [15]:
i = 1

Логика должна быть следующей: задаем системе номер клиента, она выдает для него рекомендации

In [16]:
def getting_closes(i: int):
    """Функция выдает наиболее похожих пользователей для заданного пользователя i"""
    if not i in all_users:
        return (-1, f"Пользователя {i} не существует")
    reco_df = cosine_dim.loc[((cosine_dim.x == all_users[i]) | (cosine_dim.y == all_users[i]))].sort_values('cosine', ascending=False)
    quant = reco_df.quantile(0.90).cosine
    reco_df.loc[reco_df.cosine >= quant]
    result = reco_df[reco_df['cosine'] >= quant]
    return result

In [40]:
closes_val = getting_closes(int(input('Введите id пользователя для получения рекомендаций: ')))
closes_val

,x,y,cosine
47,0,25581,0.982789
120,0,19022,0.980321
76,0,58950,0.963732
151,0,28052,0.961028
122,0,10873,0.956637
6,0,16512,0.955926
44,0,17364,0.953578
221,0,21572,0.951216
295,0,7067,0.951160
146,0,44213,0.950606


Далее берем и соединяем все книги из рекомендованых пользователей. переменная read_by_user

In [3]:
book_id = pd.read_csv('datasets/books/book_id_map.csv')
book_title = pd.read_csv('datasets/books/book_titles.csv')
call_meta = pd.read_csv('datasets/books/collaborative_book_metadata.csv')
call_book = pd.read_csv('datasets/books/collaborative_books_df.csv')
user_id = pd.read_csv('datasets/books/user_id_map.csv')

In [17]:
with open("read_by_user.pickle", "rb") as file:
    read_by_user = pickle.load(file)

In [21]:
def getting_recommendations(i: int):
    """Выдает рекомендованные id книг для пользователя i"""
    if not i in all_users:
        return (-1, f"Пользователя {i} не существует")
    read = read_by_user[i]
    closes_val = getting_closes(i)['y']
    reco_book = []
    for j in range(len(closes_val)):
            reco_book.extend(read_by_user[closes_val.iloc[j]])
    return set(reco_book)

In [26]:
all_users

[0,
 1,
 33,
 55,
 91,
 101,
 114,
 125,
 145,
 154,
 158,
 164,
 168,
 170,
 186,
 188,
 191,
 196,
 198,
 199,
 200,
 208,
 227,
 228,
 238,
 239,
 243,
 251,
 261,
 265,
 279,
 284,
 300,
 307,
 313,
 315,
 317,
 328,
 337,
 349,
 360,
 368,
 379,
 401,
 416,
 427,
 431,
 437,
 456,
 459,
 470,
 471,
 473,
 479,
 484,
 487,
 495,
 511,
 513,
 532,
 537,
 541,
 545,
 546,
 548,
 562,
 576,
 587,
 589,
 606,
 607,
 618,
 637,
 640,
 684,
 696,
 699,
 720,
 723,
 729,
 736,
 745,
 754,
 761,
 775,
 798,
 807,
 813,
 815,
 822,
 827,
 835,
 837,
 843,
 845,
 852,
 856,
 870,
 872,
 875,
 877,
 893,
 902,
 909,
 917,
 918,
 950,
 951,
 975,
 988,
 991,
 1001,
 1003,
 1006,
 1016,
 1034,
 1044,
 1049,
 1052,
 1057,
 1070,
 1072,
 1099,
 1105,
 1107,
 1110,
 1117,
 1124,
 1147,
 1154,
 1155,
 1163,
 1195,
 1196,
 1201,
 1221,
 1222,
 1249,
 1250,
 1274,
 1285,
 1295,
 1298,
 1322,
 1327,
 1330,
 1345,
 1351,
 1353,
 1354,
 1357,
 1387,
 1392,
 1397,
 1415,
 1426,
 1430,
 1432,
 1446,
 1460

In [28]:
i = int(input("Введите id пользователя: "))
if i in all_users:
    reco_set = getting_recommendations(i) - set(read_by_user[i])
    filtered_df = call_meta[call_meta['book_id'].isin(reco_set)]['title']
    print(f"{len(reco_set)} рекомендаций для пользователя {i}")
    filtered_df.to_excel(f'reco_result/reco-{i}.xlsx')
else:
    print(f"Пользователя {i} не существует")

130 рекомендаций для пользователя 196


# Дополнительный анализ датасетов 

In [32]:
len(call_book.book_id.unique())

898

In [47]:
call_meta

,Unnamed: 0,book_id,title,image_url,url,num_pages,ratings_count,description,genre,name,book_id_mapping
0,0,5899779,Pride and Prejudice and Zombies Pride and Prej...,https://images.gr-assets.com/books/1320449653m...,https://www.goodreads.com/book/show/5899779-pr...,320,105537,The New York Times Best Seller is now a major ...,"['fantasy, paranormal', 'romance', 'fiction', ...",Jane Austen,808
1,1,872333,Blue Bloods Blue Bloods 1,https://images.gr-assets.com/books/1322281515m...,https://www.goodreads.com/book/show/872333.Blu...,302,117633,"When the Mayflower set sail in 1620, it carrie...","['young-adult', 'fantasy, paranormal', 'romanc...",Melissa de la Cruz,217
2,2,15507958,Me Before You Me Before You 1,https://images.gr-assets.com/books/1357108762m...,https://www.goodreads.com/book/show/15507958-m...,369,609327,Louisa Clark is an ordinary young woman living...,"['romance', 'fiction']",Jojo Moyes,385
3,3,66559,Sharp Objects,https://images.gr-assets.com/books/1423241485m...,https://www.goodreads.com/book/show/66559.Shar...,254,208394,"Fresh from a brief stay at a psych hospital, r...","['mystery, thriller, crime', 'fiction']",Gillian Flynn,192
4,4,7235533,The Way of Kings The Stormlight Archive 1,https://images.gr-assets.com/books/1507307887m...,https://www.goodreads.com/book/show/7235533-th...,1007,151473,"Speak again the ancient oaths,\nLife before de...","['fantasy, paranormal', 'fiction']",Brandon Sanderson,873
...,...,...,...,...,...,...,...,...,...,...,...
91,91,10600242,How to Be a Woman,https://images.gr-assets.com/books/1405909800m...,https://www.goodreads.com/book/show/10600242-h...,312,72029,Though they have the vote and the Pill and hav...,"['non-fiction', 'history, historical fiction, ...",Caitlin Moran,356
92,92,9516,Persepolis The Story of a Childhood Persepolis 1,https://images.gr-assets.com/books/1425871473m...,https://www.goodreads.com/book/show/9516.Perse...,153,119470,"A New York TimesNotable Book\nA Time Magazine""...","['comics, graphic', 'non-fiction', 'history, h...",Mattias Ripa,708
93,93,307791,The City of Ember Book of Ember 1,https://images.gr-assets.com/books/1397931596m...,https://www.goodreads.com/book/show/307791.The...,270,202775,"Many hundreds of years ago, the city of Ember ...","['young-adult', 'fantasy, paranormal', 'fictio...",Jeanne DuPrau,175
94,94,37442,Wicked The Life and Times of the Wicked Witch ...,https://images.gr-assets.com/books/1437733293m...,https://www.goodreads.com/book/show/37442.Wicked,409,508863,When Dorothy triumphed over the Wicked Witch o...,"['fantasy, paranormal', 'fiction', 'young-adul...",Douglas Smith,374


In [19]:
all_books = []
for sublist in read_by_user.values():
    all_books.extend(sublist)

In [23]:
len(set(all_books))

898

In [14]:
len(call_meta['book_id'].unique())

96